### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/feat/new_tutors/examples/gobot_extended_tutorial_RU.ipynb).

# Целеориентированный бот на DeepPavlov

Данный тюториал описывает, как построить целеориентированную диалоговую систему с использованием фреймворка DeepPavlov на примере ресторанного домена. Тюториал покрывает следующие шаги:

0. [Подготовка Данных](#0.-Подготовка-Данных)
1. [Построение Базы Данных](#1.-Построение-Базы-Данных)
2. [Построение Заполнителя Слотов](#2.-Построение-Заполнителя-Слотов)
3. [Построение и Обучение Бота](#3.-Построение-и-Обучение-Бота)
4. [Взаимодействие с Ботом](#4.-Взаимодействие-с-Ботом)

Пример финальной модели доступен как бот в Телеграм:

![gobot_example.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_example.png?raw=1)

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!pip install deeppavlov
!python -m deeppavlov install gobot_simple_dstc2

## 0. Подготовка Данных

В данном тюториале мы строим чат-бота для бронирования ресторана. Чтобы обучить чат-бота мы используем датасет [Dialogue State Tracking Challenge 2 (DSTC-2)](http://camdial.org/~mh521/dstc/). DSTC-2 содержит разговоры людей в системе бронирования ресторанов, размеченные по слотам и диалоговым актам. Эта разметка будет использована для обучения нейросети (dialogue policy network).

Для начала, давайте заглянем в выбранный датасет.

In [ ]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

In [ ]:
!ls my_data

Обучающая, валидационная и тестовые выборки хранятся в json файлах (`simple-dstc2-trn.json`, `simple-dstc2-val.json` и `simple-dstc2-tst.json`):

In [ ]:
!head -n 101 my_data/simple-dstc2-trn.json

Чтобы итерироваться по батчам предобработанного DSTC-2, нам необходимо импортировать `DatasetIterator` из `DeepPavlov`.

In [ ]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

Теперь мы можем итерироваться по батчам предобработанных диалогов DSTC-2:

In [ ]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

В реальной жизни качественное аннотирование данных стоит дорого. Для приближения нашего тюториала к реалистичному сценарию использования, используем только 50 диалогов.

In [ ]:
!cp my_data/simple-dstc2-trn.json my_data/simple-dstc2-trn.full.json

In [ ]:
import json

NUM_TRAIN = 50

with open('my_data/simple-dstc2-trn.full.json', 'rt') as fin:
    data = json.load(fin)
with open('my_data/simple-dstc2-trn.json', 'wt') as fout:
    json.dump(data[:NUM_TRAIN], fout, indent=2)
print(f"Train set is reduced to {NUM_TRAIN} dialogues (out of {len(data)}).")

## 1. Построение Базы Данных

### Построение базы данных ресторанов

Чтобы выполнить задачу бронирования ресторанов чат-бот должен иметь доступ к базе данных  ресторанов `database`. База данных `database`содержит специфичную информацию, такую как к какой кухне относится еда, ценовой диапазон, расположение ресторана и другое.

    >> database([{'pricerange': 'cheap', 'area': 'south'}])
    
    Out[1]: 
        [[{'name': 'the lucky star',
           'food': 'chinese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way cherry hinton',
           'phone': '01223 244277',
           'postcode': 'c.b 1, 7 d.y'},
          {'name': 'nandos',
           'food': 'portuguese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way',
           'phone': '01223 327908',
           'postcode': 'c.b 1, 7 d.y'}]]
           

&nbsp;
![gobot_database.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_database.png?raw=1)
&nbsp;

Чат-бот должен быть обучен делать запросы к API. Для этого тренировочный датасет содержит словарь с ключами`"db_result"`. Он аннотирует диалоговые шаги (dialog turn - ровно 1 пара реплик собеседников: реплика одного участника диалога и следующая за ней реплика другого участника диалога), когда система делает запрос к API базы данных. Результирующее значение хранится в  `"db_result"`.

In [ ]:
!head -n 78 my_data/simple-dstc2-trn.json | tail +51

Передадим в `primary_keys`  лист названий слотов, которые имеют уникальные значения для различных элементов базы. Для случая базы данных ресторанов DSTC-2 основной слот - это название ресторана.

In [ ]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["name"],
                           save_path="my_bot/db.sqlite")

Давайте найдем все результирующие значения запроса к API базы данных `"db_result"` и добавим их в нашу базу данных ресторанов:

In [ ]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

### Взаимодействие с базой данных

Тепреь можно попробовать сделать различные запросы к нашей базе данных ресторанов:

In [ ]:
database([{'pricerange': 'cheap', 'area': 'south'}])

In [ ]:
!ls my_bot

## 2. Построение Заполнителя Слотов

Заполнитель слотов `Slot Filler` - это компонента, которая находит знаечния слотов в реплике пользователя:

    >> slot_filler(['I would like some chineese food'])
    
    Out[1]: [{'food': 'chinese'}]


&nbsp;
![gobot_slotfiller.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_slotfiller.png?raw=1)
&nbsp;

Чтобы имплементировать `Slot Filler`, нам необхолимо обозначить:

 - типа слотов (**slot types**),
 - все возможные значения слотов (**slot values**),
 - также хорошо иметь примеры упоминания каждого значения каждого слота.
 
В данном тюториале разметка типов слотов `slot types` и значений слотов `slot values` должна быть определена в файле `slot_vals.json` в следующем формате:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                
Давайте используем простой не обучаемый заполнитель слотов, основанный на [расстоянии Левенштейна](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0).

In [ ]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

In [ ]:
!ls my_bot/slotfill

Взглянем на некоторые типы слотов `slot types` и значения слотов `slot values`.

In [ ]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

Проверим точность нашего простого заполнителя слотов  на датасете DSTC-2.

In [ ]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

slotfill_config = read_json(configs.ner.slotfill_simple_dstc2_raw)

Возьмем [конфигурационный файл для оригинального заполнителя слотов DSTC-2](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/ner/slotfill_dstc2_raw.json) из библиотеки `DeepPavlov` и изменим переменные, отвечающие за пути к данным на текущие значения для нашего заполнителя слотов:

In [ ]:
slotfill_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

Подсчитаем метрики.

In [ ]:
from deeppavlov import evaluate_model

slotfill = evaluate_model(slotfill_config);

We've got slot accuracy of **93% on valid** set and **95% on test** set.

Поднимем модель заполнителя слотов `Slot Filler`  из конфигурационного файла `DeepPavlov`.

In [ ]:
from deeppavlov import build_model

slotfill = build_model(slotfill_config)

Протестируем модель.

In [ ]:
slotfill(['i want cheap chinee food'])

Сохранение конфигурационного файла заполнителя слотов на диск (данный путь потребуется позднее).

In [ ]:
import json

json.dump(slotfill_config, open('my_bot/slotfill_config.json', 'wt'))

In [ ]:
!ls my_bot

## 3. Построение и Обучение Бота

### Политика/структура диалога (Dialogue policy) и шаблоны ответов

Модуль dialog policy чат-бота принимает решение, какое действие будет предпринято в текущем состоянии диалога. Dialog policy в нашем чат-боте будет имплементировано как рекуррентная нейронная сеть (recurrent neural network, recurrency over user utterances), за которой следует полносвязный слой с softmax активацией. Нейросеть класифицирует входную реплику от пользователя в одно из предопределенных действия системы (определяет, какое действие требует от чат-бота данная реплика пользователя). Примеры возможных действий: поздороваться, запросить локацию пользователя или сделать запрос к базе данных. 

![gobot_policy.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_policy.png?raw=1)

Все действия, доступные для системы, должны быть перечислены в файле `simple-dstc2-templates.txt`. Каждому действию должен быть сопоставлен шаблон для соответствующего ответа системы.

![gobot_templates.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_templates.png?raw=1)

Шаблоны для ответов должны быть в формате  `<act>TAB<template>`, где `<act>` - это название действия и `<template>` - это соответствующий ответ. Шаблонный ответ может также содержать названия типов слотов, а каждый тип слота `#slot_type` будет заполнен значением слота для данного диалогового состояния.

In [ ]:
!head -n 10 my_data/simple-dstc2-templates.txt

По сути  dialogue policy модуль решает задачу классификации, где набор классов определен в `simple-dstc2-templates.txt`. Следовательно, чтобы обучить нейросеть модуля dialogue policy, нам необходимо сопоставить действие для каждого шага в диалогах обучающей выборки. Датасет DSTC-2 содержит в словаре каждого примера ключ `"act"`, который содержит название действия, требуемого текущей репликой. Рассмотрим пример обучающих данных для нейросети  dialogue policy.

In [ ]:
!head -n 24 my_data/simple-dstc2-trn.json

Тепреь мы можем собрать полный пайплайн обработки данных для ресторанного чат-бота.

Для начала, возьмем  [конфигурационный файл для простого бота DSTC2](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot/gobot_simple_dstc2.json) ([больше конфигурационных файлов доступно здесь](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot)) из `DeepPavlov` и изменим части, отвечающие за:
- эмбеддинги, 
- базу данных,
- заполнитель слотов,
- шаблоны,
- пути загрузки/сохранения модели и данных.

Загрузим чат-бота:

In [ ]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_simple_dstc2)

Возьмем bag-of-words эмбеддер:

In [ ]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Впишем использованей нашей базы данных (мы взяли всего 50 диалогов!):

In [ ]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'primary_keys': ["name"],
    'save_path': 'my_bot/db.sqlite'
}

Используем заполнитель слотов на расстоянии Левенштейна :

In [ ]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_config.json'

Для обработки значений слотов во всем диалоге, мы сначала будем находить значения слотов в последней реплике, а потом используем модуль `tracker`, который обновляет текущее значение слотов в состоянии диалога (dialogue state):

In [ ]:
gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = ['pricerange', 'this', 'area', 'food']

Внесем используемые шаблоны в пайплайн:

In [ ]:
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'my_data/simple-dstc2-templates.txt'

Обозначим  пути к train/valid/test данным и пути к сохранению нашего чат-бота:

In [ ]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'
gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

Пайплайн получившейся диалоговой системы выглядит следующим образом:

    
![gobot_pipeline.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_pipeline.png?raw=1)

### Обучение нейросети dialogue policy

In [ ]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 8 # batch size
gobot_config['train']['max_batches'] = 100 # maximum number of training batches
gobot_config['train']['log_on_k_batches'] = 20
gobot_config['train']['val_every_n_batches'] = 300 # evaluate on full 'valid' split each n batches
gobot_config['train']['log_every_n_batches'] = 50 # evaluate on 20 batches of 'train' split every n batches

train_model(gobot_config);

Обучение на 50 диалогах занимает 5-20 минут в зависимости от использумых ресурсов. Обучение на 1000 диалогов займет 10-30 минут.

См. страничку документации `DeepPavlov` [с конфигурационными файлами](http://docs.deeppavlov.ai/en/master/intro/configuration.html) для более сложных пайплайнов обучения.

### Evaluation of training

Вычисление **точности**  обученного чат-бота: соответствуют ли ответы обученной системы верным ответам из выборки (полное совпадение строки).

In [ ]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

С настройками `max_batches=250`, точность на валидационной выборке `= 0.5`, а на тестовой `~ 0.5`.

## 4. Взаимодействие с Ботом

In [ ]:
from deeppavlov import build_model

bot = build_model(gobot_config)

In [ ]:
bot(['hi, i want to eat, can you suggest a place to go?'])

In [ ]:
bot(['i want cheap food'])

In [ ]:
bot.reset()

# Бот в Telegram

Поднять своего бота из готового конфига в Телеграм очень просто.
необходимо написать BotFather и создать своего бота, получив и скопировав его токен. Токен необходимо вписать вместо `<TOKEN>`

In [ ]:
from deeppavlov.utils.telegram import interact_model_by_telegram

interact_model_by_telegram(model_config=gobot_config, token=<TOKEN>)